# PySpark - Apply Function


In [19]:
%%html
<style>
table {float:left}
</style>

In [20]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [21]:
import os
import sys
import gc
import numpy as np

# Constant

In [22]:
USER = !whoami
USER = USER[0]

PYSPARK_PYTHON_PATH = f"/home/{USER}/venv/ml/bin/python3"
HADOOP_HOME = "/opt/hadoop/hadoop-3.2.2"
SPARK_HOME = "/opt/spark/spark-3.1.2"

#  Environemnt Variables

## Hadoop

In [23]:
os.environ['HADOOP_CONF_DIR'] = f"${HADOOP_HOME}/etc/hadoop"

In [24]:
%%bash -s "$HADOOP_HOME"
export HADOOP_CONF_DIR="$1/etc/hadoop"
ls $HADOOP_CONF_DIR | head -n 3

capacity-scheduler.xml
configuration.xsl
container-executor.cfg


## PYTHONPATH

Refer to the **pyspark** modules to load from the ```$SPARK_HOME/python/lib``` in the Spark installation.

* [PySpark Getting Started](https://spark.apache.org/docs/latest/api/python/getting_started/install.html)

> Ensure the SPARK_HOME environment variable points to the directory where the tar file has been extracted. Update PYTHONPATH environment variable such that it can find the PySpark and Py4J under SPARK_HOME/python/lib. One example of doing this is shown below:

```
export PYTHONPATH=$(ZIPS=("$SPARK_HOME"/python/lib/*.zip); IFS=:; echo "${ZIPS[*]}"):$PYTHONPATH
```

Alternatively install **pyspark** with pip or conda locally which installs the Spark runtime libararies (for standalone).

* [Can PySpark work without Spark?](https://stackoverflow.com/questions/51728177/can-pyspark-work-without-spark)

> As of v2.2, executing pip install pyspark will install Spark. If you're going to use Pyspark it's clearly the simplest way to get started. On my system Spark is installed inside my virtual environment (miniconda) at lib/python3.6/site-packages/pyspark/jars  
> PySpark has a Spark installation installed. If installed through pip3, you can find it with pip3 show pyspark. Ex. for me it is at ~/.local/lib/python3.8/site-packages/pyspark. This is a standalone configuration so it can't be used for managing clusters like a full Spark installation.

In [25]:
# os.environ['PYTHONPATH'] = "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip:/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
sys.path.extend([
    f"{SPARK_HOME}/python/lib/py4j-0.10.9-src.zip",
    f"{SPARK_HOME}/python/lib/pyspark.zip"
])

## PYSPARK_PYTHON

In [26]:
os.environ['PYSPARK_PYTHON'] = PYSPARK_PYTHON_PATH

## PySpark packages

Execute after the PYTHONPATH setup.

In [27]:
import pyspark.sql 
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col,
    lit,
    when,
    isnan,
    isnull,
    lower,
    upper,
    concat,
    udf,
    array,
    avg,
    stddev,
    to_date,
    date_format,
    months_between,
    add_months,
)

---
# Spark Session


In [28]:
from pyspark.sql import SparkSession

In [29]:
# For YARN cluster
#    .config('spark.yarn.appMasterEnv.PYSPARK_PYTHON', f"/home/{USER}/venv/ml/bin/python3")\

spark = SparkSession.builder\
    .master('yarn') \
    .config('spark.submit.deployMode', 'client') \
    .config('spark.debug.maxToStringFields', 100) \
    .config('spark.executor.memory', '2g') \
    .config('spark.yarn.executorEnv.PYSPARK_PYTHON', f"/home/{USER}/venv/ml/bin/python3")\
    .getOrCreate()

2022-02-24 08:37:35,631 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-02-24 08:37:35,936 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [30]:
NUM_CORES = 4
NUM_PARTITIONS = 3

spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set('spark.sql.legacy.timeParserPolicy', 'LEGACY')

---

# Templates

## CSV
```
path_to_csv = "flight/flightData.csv"
date_format = "yyyy-MM-dd"


df = spark.read\
    .option("compression", "none")\
    .option("header", True)\
    .option("sep", ',')\
    .option("nullValue", np.nan)\
    .option("inferSchema", True)\
    .option("dateFormat", date_format)\
    .csv(path_to_csv)\
    .withColumn("date", to_date(col("date"), date_format))\
    .withColumn(
        "direction", 
        when(lower(col("from")) == "uk",  1)
        .when(lower(col("to"))  == "uk", -1)
        .otherwise(0)
    )\
    .withColumnRenamed("direction", "flight_direction")

df.printSchema()
df.show(5, truncate=False)
```

## Create View
```
df_table_name = "flight
df.createOrReplaceTempView(df_table_name)
```

## SQL
```
query = """
SELECT
    F(Z) OVER (PARTITION BY X ORDER BY Y) AS FZ
FROM
ORDER BY 
"""
spark.sql(query).show()
spark.sql(query).explain(True)
```

---
# Stop Spark Session

In [35]:
spark.stop()



# Cleanup

In [36]:
del spark
gc.collect()

1547